In [12]:
import pyreadr
import numpy as np

result = pyreadr.read_r("SMRi ordered - Brazil")   # também lê .RData
obj = next(iter(result.values()))        # pega o único objeto do .rds

# se for um data frame R, vira um pandas.DataFrame
import pandas as pd
df = obj if isinstance(obj, pd.DataFrame) else None
df

,C349_F,C509_F,C349_M,C509_M,C169_F,C61_M,C539_F,C159_F,C169_M,C189_F,...,C348_F,C259_F,C158_F,C710_F,C910_F,C508_M,C329_F,C229_M,C920_M,C348_M
0,1.123807,0.661231,0.986634,0.707814,1.221911,0.742698,1.529891,0.683831,1.314676,0.483973,...,0.129755,0.741873,0.324420,0.945515,1.155595,0.489597,0.708779,0.997034,1.632688,0.421424
1,0.443441,0.610168,0.448988,1.158253,0.574836,0.669648,1.016424,0.642775,0.857079,1.139936,...,0.686543,0.676663,0.377033,0.404505,0.856310,0.394149,1.116109,0.785896,0.856177,0.488565
2,0.879448,0.449045,0.890769,0.604989,1.166260,0.811681,0.525437,0.498478,0.860090,0.458524,...,1.018784,0.343900,0.199252,0.557964,0.717085,1.581079,0.605687,0.927671,0.976468,0.276429
3,1.020218,0.422304,1.001927,0.662585,0.601234,1.225795,0.519147,0.668390,0.739988,0.385682,...,0.508094,0.407007,0.222651,0.954067,0.589336,0.235744,0.137473,2.151896,0.491282,0.296950
4,0.443073,0.354335,0.906988,0.692725,0.575117,0.642738,0.642628,0.642295,0.661374,0.714549,...,0.998982,0.675574,0.377434,1.038157,0.865526,0.377844,0.417678,0.759698,0.842549,0.472608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,0.795859,0.878455,0.941837,0.501845,0.541944,0.668475,0.851317,0.613098,0.597926,0.443320,...,0.793249,1.046697,0.670988,1.020633,0.796339,0.500193,0.575162,0.621490,1.009267,0.408773
553,0.736944,0.657660,0.301150,0.686377,0.532942,0.702059,0.835282,0.613446,0.616894,1.295596,...,0.285469,0.243286,0.538304,0.565433,0.770437,0.764510,0.601369,0.813694,0.542717,0.372696
554,0.574846,0.596009,0.625308,0.505195,0.645011,0.514871,0.851198,0.707966,0.449321,0.463246,...,0.368908,0.168814,0.558871,0.578338,0.502741,0.387714,0.233878,0.300325,0.583313,0.472506
555,1.210127,0.824201,0.832557,1.017706,0.747300,1.087854,1.233877,1.062437,0.534971,0.742545,...,0.471792,1.141518,0.258733,0.956117,1.152759,0.457115,0.543120,0.724577,1.129771,0.767634


In [13]:
vars_cols = df.var()   
vars_cols.head()

C349_F    0.674739
C509_F    0.179864
C349_M    0.568566
C509_M    0.175342
C169_F    0.355013
dtype: float64

In [14]:
df_std = df.copy()

#vamos padronizar as colunas 
mu = df_std.mean(axis=0)
sd = df_std.std(axis=0, ddof=0)  # desvio padrão populacional (ddof=0)

# evita divisão por zero (coluna constante). Aqui descartamos constantes.
keep = sd > 0

# Y é o DataFrame padronizado
Y = ((df_std.loc[:, keep] - mu[keep]) / sd[keep]).fillna(0.0)
print(Y.values, Y.shape)
Y.var().head()

[[ 0.36497084 -0.18883623  0.27358684 ...  0.17555823  1.919834
  -0.54308607]
 [-0.46404909 -0.30934677 -0.44008088 ... -0.18922218 -0.1473854
  -0.44313775]
 [ 0.06722115 -0.68960224  0.14633648 ...  0.05572086  0.17285073
  -0.75892984]
 ...
 [-0.30393368 -0.34276097 -0.20603454 ... -1.02813628 -0.87380289
  -0.46704425]
 [ 0.47015061  0.19577766  0.06906562 ... -0.29516127  0.58097297
  -0.02770818]
 [-0.58646782 -0.75816981  0.17664611 ... -0.04401604  1.72514341
  -0.75674525]] (557, 30)


C349_F    1.001799
C509_F    1.001799
C349_M    1.001799
C509_M    1.001799
C169_F    1.001799
dtype: float64

In [15]:
# Agora sim, podemos fazer a SVD
from numpy.linalg import svd

# X = M S Vt
M, S, Vt = svd(Y.values, full_matrices=False)
print(M.shape, S.shape, Vt.shape)  # M:(n×q), S:(q,), Vt:(q×q)

(557, 30) (30,) (30, 30)


In [ ]:
# Gavish–Donoho para escolher K 
n, q = Y.shape

beta = min(q / n, n / q)
sigma_median = np.median(S)
omega = 0.56*beta**3 - 0.95*beta**2 + 1.82*beta + 1.43
tau = omega * sigma_median
K = np.sum(S >= tau)
print(f"n={n}, q={q}, beta={beta:.3f}, omega={omega:.3f}, tau={tau:.3f}, K={K}")

n=557, q=30, beta=0.054, omega=1.525, tau=20.351, K=6
